<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool28_Almudaya_Simple_Sandbox_for_Malware_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyshark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.5 MB/s eta 0:00:00


In [3]:
!pip install watchdog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.7 MB/s eta 0:00:00


In [4]:
!pip install psutil

In [5]:
import os
import subprocess
import time
import psutil
import pyshark
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from pathlib import Path

In [25]:
# Direktori dan file
MALWARE_DIR = Path("malware")
MALWARE_FILE = MALWARE_DIR / "malware_sample.py"
LOG_FILE = Path("logs/sandbox_log.txt")

In [6]:
# Direktori dan file
MALWARE_DIR = "malware"
MALWARE_FILE = os.path.join(MALWARE_DIR, "malware_sample.py")

# Konten kode malware sederhana
malware_code = """\
import os
import time

def create_file():
    with open("malware_test.txt", "w") as f:
        f.write("This is a test file created by malware.")

def modify_file():
    with open("malware_test.txt", "a") as f:
        f.write("\\nThis line was added by malware.")

def delete_file():
    if os.path.exists("malware_test.txt"):
        os.remove("malware_test.txt")

def main():
    create_file()
    time.sleep(2)
    modify_file()
    time.sleep(2)
    delete_file()

if __name__ == "__main__":
    main()
"""

# Membuat direktori jika belum ada
if not os.path.exists(MALWARE_DIR):
    os.makedirs(MALWARE_DIR)

# Menulis konten kode ke file malware_sample.py
with open(MALWARE_FILE, "w") as file:
    file.write(malware_code)

print(f"File {MALWARE_FILE} telah dibuat.")

File malware/malware_sample.py telah dibuat.


In [26]:
# Membuat direktori logs jika belum ada
if not os.path.exists("logs"):
    os.makedirs("logs")

# Membuat file log kosong jika belum ada
if not LOG_FILE.exists():
    with open(LOG_FILE, "w") as f:
        pass

In [8]:
# Fungsi untuk mencatat log
def log(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(f"{time.ctime()}: {message}\n")

In [9]:
# Fungsi untuk memantau proses
def monitor_process(proc):
    try:
        while proc.is_running():
            log(f"Process {proc.pid} is running")
            time.sleep(1)
    except psutil.NoSuchProcess:
        log(f"Process {proc.pid} terminated")
    except Exception as e:
        log(f"Error monitoring process: {str(e)}")

In [10]:
# Handler untuk memantau perubahan sistem file
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        log(f"File modified: {event.src_path}")
    def on_created(self, event):
        log(f"File created: {event.src_path}")
    def on_deleted(self, event):
        log(f"File deleted: {event.src_path}")

In [11]:
# Fungsi untuk menjalankan malware dalam sandbox
def run_malware():
    try:
        log("Starting malware execution")
        proc = subprocess.Popen(["python", MALWARE_FILE], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Gunakan psutil untuk memantau proses
        ps_proc = psutil.Process(proc.pid)
        monitor_process(ps_proc)

        stdout, stderr = proc.communicate()
        log(f"Malware output:\n{stdout.decode()}\n{stderr.decode()}")

    except Exception as e:
        log(f"Error running malware: {str(e)}")

In [12]:
# Fungsi untuk menangkap lalu lintas jaringan
def capture_network_traffic(duration=60):
    capture = pyshark.LiveCapture(interface='eth0')
    capture.sniff(timeout=duration)
    for packet in capture.sniff_continuously(packet_count=10):
        log(f"Packet captured: {packet}")

In [ ]:
!apt-get install tshark

In [16]:
import asyncio

In [20]:
async def capture_network_traffic_async(duration=60):
    capture = pyshark.LiveCapture(interface='eth0')
    await capture.sniff(timeout=duration)
    async for packet in capture.sniff_continuously(packet_count=10):
        log(f"Packet captured: {packet}")

In [ ]:
import asyncio

async def capture_network_traffic_async(duration=60): # Define the function first
    capture = pyshark.LiveCapture(interface='eth0')
    await capture.sniff(timeout=duration)
    async for packet in capture.sniff_continuously(packet_count=10):
        log(f"Packet captured: {packet}")

if __name__ == "__main__":
    # Memulai pemantauan sistem file
    event_handler = FileChangeHandler()
    observer = Observer()
    observer.schedule(event_handler, path=MALWARE_DIR, recursive=True)
    observer.start()

    # Use asyncio.create_task to schedule the coroutine within the existing loop
    asyncio.create_task(capture_network_traffic_async())

    # Menjalankan malware
    run_malware()

    # Menghentikan pemantauan sistem file
    observer.stop()
    observer.join()

    log("Sandbox completed")